In [80]:
import pandas as pd
import country_converter as coco

# Initialize converter
cc = coco.CountryConverter()

# Access the internal country table
df = cc.data

# Extract unique IOC codes, dropping NaNs
ioc_list = df['IOC'].dropna().unique().tolist()


# Convert to ISO3
iso3_list = cc.convert(names=ioc_list, src='IOC', to='ISO3')

# Also get the full country names (optional but helpful)
full_names = cc.convert(names=ioc_list, src='IOC', to='name_short')

# Put everything in a DataFrame
df_check = pd.DataFrame({
    'IOC': ioc_list,
    'ISO3': iso3_list,
    'Country Name': full_names
})

print(df_check)


     IOC ISO3    Country Name
0    AFG  AFG     Afghanistan
1    ALB  ALB         Albania
2    ALG  DZA         Algeria
3    ASA  ASM  American Samoa
4    AND  AND         Andorra
..   ...  ...             ...
201  VEN  VEN       Venezuela
202  VIE  VNM         Vietnam
203  YEM  YEM           Yemen
204  ZAM  ZMB          Zambia
205  ZIM  ZWE        Zimbabwe

[206 rows x 3 columns]


# Exercise 1
Write a Python function that can be used to query data from the Worldbank Indicator API. Your
function should:
• take the following input parameters: indicators, countries, and years.
• return a Pandas DataFrame of the queried data
• have a docstring that explains what the function does, what the input parameters are, and what
the output is
• minimize the number of API calls necessary to retrieve the data

In [82]:
import requests
import pandas as pd

def worldbank_data(indicators, countries, years):
    """
    Fetch data from the World Bank Indicator API for specified indicators, countries, and years.

    Parameters:
        indicators (list): A list of indicator codes (e.g. ["SP.POP.TOTL"]).
        countries (list or str): A list of ISO-2 country codes (e.g. ["DE", "FR"]) or the string "all".
        years (list or tuple): A single year [2012] or a range like (2010, 2020).

    Returns:
        pd.DataFrame: A DataFrame containing the country, indicator, year, and value.
    """

    if isinstance(countries, str):
        country_str = countries
    elif isinstance(countries, list) and countries == ["all"]:
        country_str = "all"
    else:
        country_str = ';'.join(countries)

    # Handle year
    if len(years) == 2:
        date_str = f"{years[0]}:{years[1]}"
    else:
        date_str = str(years[0])

    all_records = []

    # Loop through indicators if using "all" countries
    for indicator in indicators:
        url = f"https://api.worldbank.org/v2/country/{country_str}/indicator/{indicator}"
        params = {
            'date': date_str,
            'format': 'json',
            'per_page': 10000
        }

        response = requests.get(url, params=params)
        if response.status_code != 200:
            raise Exception(f"Failed to fetch data: {response.status_code} for {indicator}")

        try:
            data_json = response.json()
        except ValueError:
            raise Exception("Response is not valid JSON")

        if not isinstance(data_json, list) or len(data_json) < 2:
            print("No data returned or invalid format. Here's the response:")
            print(data_json)
            continue  # Skip this indicator

        data = data_json[1]
        records = [{
            'country': entry['country']['value'],
            'country_code': entry['countryiso3code'],
            'date': int(entry['date']),
            'indicator': entry['indicator']['id'],
            'value': entry['value']
        } for entry in data]

        all_records.extend(records)

    return pd.DataFrame.from_records(all_records)


Demonstrate that your function works by querying the following data (codes are provided in parentheses):

1) The total population (SP.POP.TOTL) of Germany (DE) and France (FR) between 2015 and 2020.

In [83]:
worldbank_data(["SP.POP.TOTL"],["DE","FR"],(2015,2020))

,country,country_code,date,indicator,value
0,Germany,DEU,2020,SP.POP.TOTL,83160871
1,Germany,DEU,2019,SP.POP.TOTL,83092962
2,Germany,DEU,2018,SP.POP.TOTL,82905782
3,Germany,DEU,2017,SP.POP.TOTL,82657002
4,Germany,DEU,2016,SP.POP.TOTL,82348669
5,Germany,DEU,2015,SP.POP.TOTL,81686611
6,France,FRA,2020,SP.POP.TOTL,67601110
7,France,FRA,2019,SP.POP.TOTL,67382061
8,France,FRA,2018,SP.POP.TOTL,67158348
9,France,FRA,2017,SP.POP.TOTL,66918020


2) The total population (SP.POP.TOTL), GDP in current US$ (NY.GDP.MKTP.CD), and life expectancy in years at birth (SP.DYN.LE00.IN) of all countries (all) in 2012. Print the shape of the
resulting DataFrame and display its first 10 rows.

In [84]:
df = worldbank_data(["SP.POP.TOTL","NY.GDP.MKTP.CD","SP.DYN.LE00.IN"],"all",[2012])#.head(10)

In [85]:
df.head(10)

,country,country_code,date,indicator,value
0,Africa Eastern and Southern,AFE,2012,SP.POP.TOTL,5.596100e+08
1,Africa Western and Central,AFW,2012,SP.POP.TOTL,3.853603e+08
2,Arab World,ARB,2012,SP.POP.TOTL,3.810302e+08
3,Caribbean small states,CSS,2012,SP.POP.TOTL,2.893251e+06
4,Central Europe and the Baltics,CEB,2012,SP.POP.TOTL,1.039281e+08
5,Early-demographic dividend,EAR,2012,SP.POP.TOTL,3.058869e+09
6,East Asia & Pacific,EAS,2012,SP.POP.TOTL,2.249081e+09
7,East Asia & Pacific (excluding high income),EAP,2012,SP.POP.TOTL,2.006566e+09
8,East Asia & Pacific (IDA & IBRD countries),TEA,2012,SP.POP.TOTL,1.981383e+09
9,Euro area,EMU,2012,SP.POP.TOTL,3.404478e+08


In [86]:
df.shape

(798, 5)

In [87]:
# Use the pivot_table function to spread the row of the dataset to contain the population, GDP and life expectancy for each country.
df_pivot = df.pivot_table(
    index=["country", "country_code", "date"],
    columns="indicator",
    values="value"
)

# Remove the "indicator" label from columns to avoid confusion
df_pivot.columns.name = None

# Reset the index
df_pivot = df_pivot.reset_index()

In [88]:
df_pivot

,country,country_code,date,NY.GDP.MKTP.CD,SP.DYN.LE00.IN,SP.POP.TOTL
0,Afghanistan,AFG,2012,1.990733e+10,61.735000,3.056003e+07
1,Africa Eastern and Southern,AFE,2012,9.529986e+11,60.249512,5.596100e+08
2,Africa Western and Central,AFW,2012,7.399381e+11,55.292706,3.853603e+08
3,Albania,ALB,2012,1.231983e+10,78.084000,2.900401e+06
4,Algeria,DZA,2012,2.271437e+11,74.609000,3.764617e+07
...,...,...,...,...,...,...
260,West Bank and Gaza,PSE,2012,1.220840e+10,73.722000,3.979998e+06
261,World,WLD,2012,7.570585e+13,71.262232,7.175816e+09
262,"Yemen, Rep.",YEM,2012,3.540133e+10,67.569000,2.843365e+07
263,Zambia,ZMB,2012,2.550306e+10,58.704000,1.491363e+07


In [89]:
df_pivot.rename(columns={
    'NY.GDP.MKTP.CD': 'GDP',
    'SP.DYN.LE00.IN': 'Life_Expectancy',
    'SP.POP.TOTL': 'Population'
}, inplace=True)


In [90]:
df_pivot

,country,country_code,date,GDP,Life_Expectancy,Population
0,Afghanistan,AFG,2012,1.990733e+10,61.735000,3.056003e+07
1,Africa Eastern and Southern,AFE,2012,9.529986e+11,60.249512,5.596100e+08
2,Africa Western and Central,AFW,2012,7.399381e+11,55.292706,3.853603e+08
3,Albania,ALB,2012,1.231983e+10,78.084000,2.900401e+06
4,Algeria,DZA,2012,2.271437e+11,74.609000,3.764617e+07
...,...,...,...,...,...,...
260,West Bank and Gaza,PSE,2012,1.220840e+10,73.722000,3.979998e+06
261,World,WLD,2012,7.570585e+13,71.262232,7.175816e+09
262,"Yemen, Rep.",YEM,2012,3.540133e+10,67.569000,2.843365e+07
263,Zambia,ZMB,2012,2.550306e+10,58.704000,1.491363e+07


State how many API calls your function makes for (a) and (b) respectively.

1) The first one was once
2) The second one was 3 times, because three indicators was provided

# Exercise 2
The file medal_table_2012.csv contains information about the number of medals won by each country
at the Olympic Games 2012. (It probably looks similar to the medal table that you calculated in the
first part of the project. Small differences are possible, but the overall structure should be the same.)


(a) Preprocess both the medal table data and the Worldbank data retrieved in exercise 1 (b) and
combine the two datasets suitably into one tidy dataset. The final dataset should be such that it
allows you to answer the following exercises. Explain your actions and decisions in a few sentences.

In [91]:
medal_table_2012 = pd.read_csv("medal_table_2012.csv")

In [92]:
medal_table_2012

,rank,country_code,country,gold,silver,bronze
0,1,USA,United States,46,28,30
1,2,CHN,People's Republic of China,38,31,22
2,3,GBR,Great Britain,29,17,19
3,4,RUS,Russian Federation,20,20,27
4,5,KOR,Republic of Korea,13,9,8
...,...,...,...,...,...,...
80,81,KSA,Kingdom of Saudi Arabia,0,0,1
81,82,KUW,Kuwait,0,0,1
82,83,MAR,Morocco,0,0,1
83,84,TJK,Tajikistan,0,0,1


In [93]:
medal_table_2012.describe()

,rank,gold,silver,bronze
count,85.000000,85.000000,85.000000,85.000000
mean,43.000000,3.482353,3.470588,4.070588
std,24.681302,7.512865,5.927314,5.830520
min,1.000000,0.000000,0.000000,0.000000
25%,22.000000,0.000000,0.000000,1.000000
50%,43.000000,1.000000,1.000000,2.000000
75%,64.000000,3.000000,4.000000,4.000000
max,85.000000,46.000000,31.000000,30.000000


In [94]:
df_pivot

,country,country_code,date,GDP,Life_Expectancy,Population
0,Afghanistan,AFG,2012,1.990733e+10,61.735000,3.056003e+07
1,Africa Eastern and Southern,AFE,2012,9.529986e+11,60.249512,5.596100e+08
2,Africa Western and Central,AFW,2012,7.399381e+11,55.292706,3.853603e+08
3,Albania,ALB,2012,1.231983e+10,78.084000,2.900401e+06
4,Algeria,DZA,2012,2.271437e+11,74.609000,3.764617e+07
...,...,...,...,...,...,...
260,West Bank and Gaza,PSE,2012,1.220840e+10,73.722000,3.979998e+06
261,World,WLD,2012,7.570585e+13,71.262232,7.175816e+09
262,"Yemen, Rep.",YEM,2012,3.540133e+10,67.569000,2.843365e+07
263,Zambia,ZMB,2012,2.550306e+10,58.704000,1.491363e+07


In [97]:
""" Before Merging the data, we nned to find a common column and since the country_code and country names are using different naming convention,
 the IOC countryc country codes used in the olympics data has to be converted ISO3 used in the world bank data"""

import country_converter as coco

# Initialize converter
cc = coco.CountryConverter()

# Access the internal country table
df = cc.data

# Extract unique IOC codes, dropping NaNs
ioc_list = df['IOC'].dropna().unique().tolist()
ioc_list


['AFG',
 'ALB',
 'ALG',
 'ASA',
 'AND',
 'ANG',
 'ANT',
 'ARG',
 'ARM',
 'ARU',
 'AUS',
 'AUT',
 'AZE',
 'BAH',
 'BRN',
 'BAN',
 'BAR',
 'BLR',
 'BEL',
 'BIZ',
 'BEN',
 'BER',
 'BHU',
 'BOL',
 'BIH',
 'BOT',
 'BRA',
 'IVB',
 'BRU',
 'BUL',
 'BUR',
 'BDI',
 'CPV',
 'CAM',
 'CMR',
 'CAN',
 'CAY',
 'CAF',
 'CHA',
 'CHI',
 'CHN',
 'COL',
 'COM',
 'CGO',
 'COK',
 'CRC',
 'CIV',
 'CRO',
 'CUB',
 'CYP',
 'CZE',
 'DEN',
 'DJI',
 'DMA',
 'DOM',
 'COD',
 'ECU',
 'EGY',
 'ESA',
 'GEQ',
 'ERI',
 'EST',
 'SWZ',
 'ETH',
 'FIJ',
 'FIN',
 'FRA',
 'GAB',
 'GAM',
 'GEO',
 'GER',
 'GHA',
 'GRE',
 'GRN',
 'GUM',
 'GUA',
 'GUI',
 'GBS',
 'GUY',
 'HAI',
 'HON',
 'HKG',
 'HUN',
 'ISL',
 'IND',
 'INA',
 'IRI',
 'IRQ',
 'IRL',
 'ISR',
 'ITA',
 'JAM',
 'JPN',
 'JOR',
 'KAZ',
 'KEN',
 'KIR',
 'KOS',
 'KUW',
 'KGZ',
 'LAO',
 'LAT',
 'LBN',
 'LES',
 'LBR',
 'LBA',
 'LIE',
 'LTU',
 'LUX',
 'MKD',
 'MAD',
 'MAW',
 'MAS',
 'MDV',
 'MLI',
 'MLT',
 'MHL',
 'MTN',
 'MRI',
 'MEX',
 'FSM',
 'MDA',
 'MON',
 'MGL',
 'MNE',


In [104]:


# Convert to ISO3
iso3_list = cc.convert(names=ioc_list, src='IOC', to='ISO3')

# Also get the full country names (optional but helpful)
full_names = cc.convert(names=ioc_list, src='IOC', to='name_short')

# Put everything in a DataFrame
df_check = pd.DataFrame({
    'country_code': ioc_list,
    'ISO3': iso3_list,
    'Country_Name': full_names
})


print(df_check)


    country_code ISO3    Country_Name
0            AFG  AFG     Afghanistan
1            ALB  ALB         Albania
2            ALG  DZA         Algeria
3            ASA  ASM  American Samoa
4            AND  AND         Andorra
..           ...  ...             ...
201          VEN  VEN       Venezuela
202          VIE  VNM         Vietnam
203          YEM  YEM           Yemen
204          ZAM  ZMB          Zambia
205          ZIM  ZWE        Zimbabwe

[206 rows x 3 columns]


In [109]:
# Next is to left merge the df_check to the medals table
df_1 = pd.merge(medal_table_2012, df_check, how="left", on="country_code")

In [110]:
df_2 = df_1.drop(columns= ["country_code","country"]).rename(columns = {'ISO3':'country_code'})

In [113]:
df_2 = df_2[["rank", "Country_Name", "country_code", "gold", "silver", "bronze"	]]

In [114]:
df_2

,rank,Country_Name,country_code,gold,silver,bronze
0,1,United States,USA,46,28,30
1,2,China,CHN,38,31,22
2,3,United Kingdom,GBR,29,17,19
3,4,Russia,RUS,20,20,27
4,5,South Korea,KOR,13,9,8
...,...,...,...,...,...,...
80,81,Saudi Arabia,SAU,0,0,1
81,82,Kuwait,KWT,0,0,1
82,83,Morocco,MAR,0,0,1
83,84,Tajikistan,TJK,0,0,1


In [ ]:
#Next is to combine the new datset with the games dataset on the year column
# Also, We are merging on the left of df_2 because we only need the world bank information of the countries in the medal table
df_1 = pd.merge(df_2, df_pivot, how="left", on="country_code")

In [116]:
df_1

,rank,Country_Name,country_code,gold,silver,bronze,country,date,GDP,Life_Expectancy,Population
0,1,United States,USA,46,28,30,United States,2012.0,1.625397e+13,78.741463,3.138777e+08
1,2,China,CHN,38,31,22,China,2012.0,8.532185e+12,76.200000,1.354190e+09
2,3,United Kingdom,GBR,29,17,19,United Kingdom,2012.0,2.707090e+12,80.904878,6.370022e+07
3,4,Russia,RUS,20,20,27,Russian Federation,2012.0,2.208294e+12,70.072195,1.433784e+08
4,5,South Korea,KOR,13,9,8,"Korea, Rep.",2012.0,1.278047e+12,80.819512,5.019985e+07
...,...,...,...,...,...,...,...,...,...,...,...
80,81,Saudi Arabia,SAU,0,0,1,Saudi Arabia,2012.0,7.418499e+11,76.090000,2.616886e+07
81,82,Kuwait,KWT,0,0,1,Kuwait,2012.0,1.740477e+11,78.395024,3.337109e+06
82,83,Morocco,MAR,0,0,1,Morocco,2012.0,1.069374e+11,71.674000,3.335524e+07
83,84,Tajikistan,TJK,0,0,1,Tajikistan,2012.0,7.633037e+09,69.130000,8.014084e+06


In [117]:
df_1.describe()

,rank,gold,silver,bronze,date,GDP,Life_Expectancy,Population
count,85.000000,85.000000,85.000000,85.000000,84.0,8.300000e+01,84.000000,8.400000e+01
mean,43.000000,3.482353,3.470588,4.070588,2012.0,8.389832e+11,74.913427,6.617667e+07
std,24.681302,7.512865,5.927314,5.830520,0.0,2.155361e+12,5.965980,2.037015e+08
min,1.000000,0.000000,0.000000,0.000000,2012.0,7.998815e+08,58.170000,1.127750e+05
25%,22.000000,0.000000,0.000000,1.000000,2012.0,4.563516e+10,71.892890,4.551675e+06
50%,43.000000,1.000000,1.000000,2.000000,2012.0,1.955907e+11,75.307707,1.120505e+07
75%,64.000000,3.000000,4.000000,4.000000,2012.0,5.478623e+11,80.376220,4.635081e+07
max,85.000000,46.000000,31.000000,30.000000,2012.0,1.625397e+13,83.450244,1.354190e+09


In [118]:
df_1 = df_1.drop(columns=["country"])

In [119]:
df_1

,rank,Country_Name,country_code,gold,silver,bronze,date,GDP,Life_Expectancy,Population
0,1,United States,USA,46,28,30,2012.0,1.625397e+13,78.741463,3.138777e+08
1,2,China,CHN,38,31,22,2012.0,8.532185e+12,76.200000,1.354190e+09
2,3,United Kingdom,GBR,29,17,19,2012.0,2.707090e+12,80.904878,6.370022e+07
3,4,Russia,RUS,20,20,27,2012.0,2.208294e+12,70.072195,1.433784e+08
4,5,South Korea,KOR,13,9,8,2012.0,1.278047e+12,80.819512,5.019985e+07
...,...,...,...,...,...,...,...,...,...,...
80,81,Saudi Arabia,SAU,0,0,1,2012.0,7.418499e+11,76.090000,2.616886e+07
81,82,Kuwait,KWT,0,0,1,2012.0,1.740477e+11,78.395024,3.337109e+06
82,83,Morocco,MAR,0,0,1,2012.0,1.069374e+11,71.674000,3.335524e+07
83,84,Tajikistan,TJK,0,0,1,2012.0,7.633037e+09,69.130000,8.014084e+06


# Explanation
1) In order to properly merge the two dataframes, I had to convert the naming convetion of the country_code used in the olympics data to the one used in the worldbank data

2) I did a left merge because I needed just the world bank data of countries in the medal table

# Exercise 2

(b) Create an alternative medal table for the 2012 Olympic Games by calculating the number of Gold,
Silver, and Bronze medals won per 10 million inhabitants. Display the 10 most successful countries
according to this alternative medal table.

In [123]:
# Avoid division by zero and drop rows with missing population data
df_1 = df_1[df_1["Population"].notna()]
df_1 = df_1[df_1['Population'] > 0]

# Calculation of medals per 10 million inhabitants
df_1["gold_per_10M"] = df_1["gold"] / (df_1["Population"] / 10_000_000)
df_1["silver_per_10M"] = df_1["silver"] / (df_1["Population"] / 10_000_000)
df_1["bronze_per_10M"] = df_1["bronze"] / (df_1["Population"] / 10_000_000)

# Total medals per 10M
df_1["Total_per_10M"] = df_1["gold_per_10M"] + df_1["silver_per_10M"] + df_1["bronze_per_10M"]

df_1

,rank,Country_Name,country_code,gold,silver,bronze,date,GDP,Life_Expectancy,Population,gold_per_10M,silver_per_10M,bronze_per_10M,Total_per_10M
0,1,United States,USA,46,28,30,2012.0,1.625397e+13,78.741463,3.138777e+08,1.465539,0.892067,0.955786,3.313393
1,2,China,CHN,38,31,22,2012.0,8.532185e+12,76.200000,1.354190e+09,0.280611,0.228919,0.162459,0.671988
2,3,United Kingdom,GBR,29,17,19,2012.0,2.707090e+12,80.904878,6.370022e+07,4.552575,2.668751,2.982721,10.204047
3,4,Russia,RUS,20,20,27,2012.0,2.208294e+12,70.072195,1.433784e+08,1.394910,1.394910,1.883128,4.672948
4,5,South Korea,KOR,13,9,8,2012.0,1.278047e+12,80.819512,5.019985e+07,2.589649,1.792834,1.593630,5.976113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,81,Saudi Arabia,SAU,0,0,1,2012.0,7.418499e+11,76.090000,2.616886e+07,0.000000,0.000000,0.382134,0.382134
81,82,Kuwait,KWT,0,0,1,2012.0,1.740477e+11,78.395024,3.337109e+06,0.000000,0.000000,2.996606,2.996606
82,83,Morocco,MAR,0,0,1,2012.0,1.069374e+11,71.674000,3.335524e+07,0.000000,0.000000,0.299803,0.299803
83,84,Tajikistan,TJK,0,0,1,2012.0,7.633037e+09,69.130000,8.014084e+06,0.000000,0.000000,1.247803,1.247803


In [125]:
# Sort by Gold_per_10M, then Silver_per_10M, then Bronze_per_10M (all descending)
alt_medal_table = df_1.sort_values(
    by=["gold_per_10M", "silver_per_10M", "bronze_per_10M"], 
    ascending=False
)
alt_medal_table

,rank,Country_Name,country_code,gold,silver,bronze,date,GDP,Life_Expectancy,Population,gold_per_10M,silver_per_10M,bronze_per_10M,Total_per_10M
50,51,Grenada,GRD,1,0,0,2012.0,7.998815e+08,74.961000,112775.0,88.672135,0.000000,0.000000,88.672135
51,52,Bahamas,BHS,1,0,0,2012.0,1.072040e+10,73.707000,375469.0,26.633357,0.000000,0.000000,26.633357
16,17,Jamaica,JAM,4,5,3,2012.0,1.480709e+10,72.405000,2772430.0,14.427776,18.034720,10.820832,43.283329
12,13,New Zealand,NZL,6,2,5,2012.0,1.762107e+11,81.407317,4408100.0,13.611306,4.537102,11.342755,29.491164
49,50,Bahrain,BHR,1,0,0,2012.0,3.196340e+10,79.622000,1208964.0,8.271545,0.000000,0.000000,8.271545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,79,Cameroon,CMR,0,0,1,2012.0,3.015506e+10,58.170000,20809529.0,0.000000,0.000000,0.480549,0.480549
80,81,Saudi Arabia,SAU,0,0,1,2012.0,7.418499e+11,76.090000,26168861.0,0.000000,0.000000,0.382134,0.382134
77,78,Afghanistan,AFG,0,0,1,2012.0,1.990733e+10,61.735000,30560034.0,0.000000,0.000000,0.327225,0.327225
82,83,Morocco,MAR,0,0,1,2012.0,1.069374e+11,71.674000,33355241.0,0.000000,0.000000,0.299803,0.299803


In [127]:


top10_alt = alt_medal_table[["Country_Name", "gold_per_10M", "silver_per_10M", "bronze_per_10M", "Total_per_10M"]].head(10)
top10_alt

,Country_Name,gold_per_10M,silver_per_10M,bronze_per_10M,Total_per_10M
50,Grenada,88.672135,0.000000,0.000000,88.672135
51,Bahamas,26.633357,0.000000,0.000000,26.633357
16,Jamaica,14.427776,18.034720,10.820832,43.283329
12,New Zealand,13.611306,4.537102,11.342755,29.491164
49,Bahrain,8.271545,0.000000,0.000000,8.271545
9,Hungary,8.064222,4.032111,6.048166,18.144499
43,Trinidad and Tobago,7.489548,7.489548,14.979097,29.958193
24,Croatia,7.041645,2.347215,4.694430,14.083290
33,Lithuania,6.693949,0.000000,10.040923,16.734872
47,Latvia,4.915650,0.000000,4.915650,9.831300


# Exercise 3
Carry out a simple supervised machine learning experiment, in which you train a model to predict the
number of Gold medals a country wins at the Olympic Games 2012 based on demographic and economic
features. Note: Since machine learning is not a focus topic of this course, you do not need to optimize
the model. Just demonstrate that you are able to apply the steps we discussed in the course and correctly
interpret the results.
(a) Train and evaluate a linear regression model: 

1. Split your data into a training and a test set.

2. Train a linear regression model using population, life expectancy and the GDP per capita of a
country as features. 

3. Evaluate the model using the root mean squared error as the performance
metric.

In [132]:
# Import libraries
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler


In [133]:
# Select relevant columns for features and target variable
features = ["Population", "Life_Expectancy", "GDP"]
target = "gold"

# Drop rows with missing values for these columns
df_cleaned = alt_medal_table.dropna(subset=features + [target])

# Define X (features) and y (target)
X = df_cleaned[features]
y = df_cleaned[target]


In [134]:
print(X)

    Population  Life_Expectancy           GDP
50    112775.0        74.961000  7.998815e+08
51    375469.0        73.707000  1.072040e+10
16   2772430.0        72.405000  1.480709e+10
12   4408100.0        81.407317  1.762107e+11
49   1208964.0        79.622000  3.196340e+10
..         ...              ...           ...
78  20809529.0        58.170000  3.015506e+10
80  26168861.0        76.090000  7.418499e+11
77  30560034.0        61.735000  1.990733e+10
82  33355241.0        71.674000  1.069374e+11
84  89510356.0        73.762000  1.955907e+11

[83 rows x 3 columns]


In [135]:
y

50    1
51    1
16    4
12    6
49    1
     ..
78    0
80    0
77    0
82    0
84    0
Name: gold, Length: 83, dtype: int64

In [136]:
# Split data into training and test sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [137]:
# Standardize the features
scaler = StandardScaler()

# Fit the scaler on the training data and transform both train and test sets
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [138]:
# Initialize and train the linear regression model
model = LinearRegression()
model.fit(X_train_scaled, y_train)


LinearRegression()

In [139]:
# Make predictions on the test set
y_pred = model.predict(X_test_scaled)

# Calculate the Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Display the RMSE
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")


Root Mean Squared Error (RMSE): 6.75


# Exercise 3
(b) Discuss your results: How do you judge the performance? What are possible reasons for this
performance? How could the model be improved?


### (b) Discussion of Results

#### **How do you judge the performance?**
The Root Mean Squared Error (RMSE) of 6.75 means that, on average, the model's predictions are off by approximately 6.75 gold medals. This gives us an indication of how well the model has generalized from the training data to the test set.

An RMSE of 6.75 suggests that the model is performing reasonably well in terms of predicting gold medals, considering that the actual number of gold medals won by some countries varies widely. For example, countries like the U.S. and China won a large number of gold medals in 2012 (around 40+), while smaller countries won fewer.

  
#### **Possible reasons for this performance:**

1. **Limited features:**
   - The model uses only three features (population, life expectancy, and GDP per capita) to predict gold medals. These are important economic and demographic indicators, but they may not capture all the factors that influence a country's Olympic success. For example, other factors like historical performance, sports infrastructure, government investment in sports, or even cultural and environmental aspects could also significantly impact medal counts.
   
2. **Non-linearity:**
   - Linear regression assumes a linear relationship between the features and the target variable. In reality, the relationship between demographic/economic factors and Olympic performance may not be purely linear. Countries with high GDP or large populations might have a disproportionate chance of winning medals, while smaller or less wealthy countries might still perform well due to other factors not captured by the model.
   
3. **Outliers:**
   - Countries with extremely high or low medal counts (e.g., the U.S. with over 40 gold medals or a country with no gold medals) can significantly skew the results. If the model does not account for these outliers properly, it might struggle to make accurate predictions, particularly for countries with average performance.



#### **How could the model be improved?**

1. **Adding more features:**
   - **Historical performance**: Including previous Olympic results or past medal counts could improve predictions.
   - **Sports infrastructure**: Metrics like the number of sports facilities, investment in sports, or funding for athletes could be helpful.
   - **Government policies**: Information about government support for sports or athlete development programs could contribute.
   - **Cultural factors**: Countries with a strong sports culture might outperform others with similar demographic and economic indicators.

2. **Using more sophisticated models:**
   - **Non-linear models**: Linear regression may not be the best fit for this problem. Models like **decision trees**, **random forests**, or **gradient boosting** could capture more complex, non-linear relationships between the features and the target variable.
   
   
4. **Handling outliers:**
   - Identifying and treating outliers in the data (e.g., using robust models, trimming outlier countries, or applying logarithmic transformations to skewed variables) can lead to better predictive performance, especially when predicting for countries with lower medal counts.


In conclusion, while the linear regression model provides a reasonable starting point, there is much room for improvement. Incorporating more features, trying non-linear models, and addressing the limitations of the data could lead to better predictions of Olympic gold medal counts.

 ***(c) Due to an unfortunate “data error”, the country Netherlands was not included in the Olympic Games data and is therefore not present in medal table.***


Use your trained machine learning model to predict the number of Gold medals the Netherlands has won in 2012, just based on their demographic and economic characteristics.

In [156]:
NLD = df_pivot[df_pivot["country_code"]== "NLD"]
NLD = NLD[features]
NLD_scaled = scaler.fit_transform(NLD)
# Predict the number of gold medals
predicted_gold_medals = model.predict(NLD_scaled)
print(f"Predicted gold medals for the Netherlands: {predicted_gold_medals[0]:.2f}")

Predicted gold medals for the Netherlands: 3.09
